In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import geopandas as gpd
from geoalchemy2 import Geometry
import shapely.wkb as wkb


In [2]:
# Leer el archivo CSV
df_incendios = pd.read_csv(r'G:\Mi unidad\OSPA\01. Tematicas\04. Incendios\01. Productos\postmodelo_icv\temporales\probabilidad_icv.csv', sep=';', encoding='utf-8')
df_deslizamientos = pd.read_csv(r'G:\Mi unidad\OSPA\01. Tematicas\03. Deslizamientos\01. Productos\postmodelo_idd\temporales\probabilidad_idd.csv', sep=';', encoding='utf-8')


In [3]:
# Crear una conexión a la base de datos
usuario = 'postgres'
contraseña = 'Septiembre0672'
host = 'localhost'
puerto = '5432'
db = 'alertas'
url_conexion = f'postgresql://{usuario}:{contraseña}@{host}:{puerto}/{db}'

In [4]:
def alertas_incendios(url, df_incendios):
    try:
        engine = create_engine(url_conexion)
        return df_incendios.to_sql('incendios', con=engine, if_exists='replace', index=False)
    except:
        engine = create_engine(url_conexion)
        # Eliminar los registros existentes de la tabla 'incendios'
        with engine.connect() as conn:
            conn.execute("DELETE FROM incendios")
        return df_incendios.to_sql('incendios', con=engine, if_exists='append', index=False)

In [5]:
alertas_incendios(url_conexion, df_incendios)

113

In [6]:
def alertas_deslizamientos(url, df_deslizamientos):
    try:
        engine = create_engine(url_conexion)
        return df_deslizamientos.to_sql('deslizamientos', con=engine, if_exists='replace', index=False)
    except:
        engine = create_engine(url_conexion)
        # Eliminar los registros existentes de la tabla 'incendios'
        with engine.connect() as conn:
            conn.execute("DELETE FROM deslizamientos")
        return df_deslizamientos.to_sql('deslizamientos', con=engine, if_exists='append', index=False)

In [7]:
alertas_deslizamientos(url_conexion, df_deslizamientos)

69

In [8]:
def vista_departamentos(url_conexion):
    
    engine = create_engine(url_conexion)
    
    create_view_sql = """
CREATE OR REPLACE VIEW departamentos_colombia AS
SELECT departamen, jsonb_build_object(
  'type',       'Feature',
  'geometry',   ST_AsGeoJSON(geom, 4)::jsonb,
  'properties', jsonb_build_object(
    'DEPARTAMENTO', departamen
  )
) AS geom
FROM public.departamentos_wgs84

"""
    try:
        with engine.connect() as conn:
            conn.execute(create_view_sql)
            return("Vista creada exitosamente.")
    except SQLAlchemyError as e:
        return(f"Error al crear la vista: {e}")

In [9]:
vista_departamentos(url_conexion)

'Vista creada exitosamente.'

In [10]:
def vista_deslizamientos(url_conexion):
    
    engine = create_engine(url_conexion)
    
    create_view_sql = """
CREATE OR REPLACE VIEW alertas_deslizamientos AS
SELECT d.*,
jsonb_build_object(
  'type', 'Feature',
  'geometry', ST_AsGeoJSON(ST_GeometryN(mg.geom, 1), 4)::jsonb,
  'properties', jsonb_build_object(
    'PROBABILIDAD_DESC', CASE
                           WHEN d."PROBABILIDAD" = 1 THEN 'BAJA'
                           WHEN d."PROBABILIDAD" = 2 THEN 'MEDIA'
                           WHEN d."PROBABILIDAD" = 3 THEN 'ALTA'
                           ELSE 'no definida' -- Opcional: para valores distintos de 1, 2 o 3
                         END,
    'PROBABILIDAD', d."PROBABILIDAD",
    'REGION', d."REGION",
    'DEPARTAMENTO', d."DEPARTAMENTO",
    'MUNICIPIO', d."MUNICIPIO",
	  'FECHA_EJECUCION', d."FECHA_EJECUCION"
  )
) AS geom
FROM deslizamientos AS d
INNER JOIN mgn_mpio_politico AS mg
ON d."COD_DANE" = mg.mpio_cdpmp;

"""
    try:
        with engine.connect() as conn:
            conn.execute(create_view_sql)
            return("Vista creada exitosamente.")
    except SQLAlchemyError as e:
        return(f"Error al crear la vista: {e}")

In [11]:
vista_deslizamientos(url_conexion)

'Vista creada exitosamente.'

In [12]:
def vista_incendios(url_conexion):
    
    engine = create_engine(url_conexion)
    
    create_view_sql = """
CREATE OR REPLACE VIEW alertas_incendios AS
SELECT i.*, jsonb_build_object(
  'type',       'Feature',
  'geometry',   ST_AsGeoJSON(ST_GeometryN(mg.geom, 1), 4)::jsonb,
  'properties', jsonb_build_object(
	 'PROBABILIDAD_DESC', CASE
                           WHEN i."PROBABILIDAD" = 1 THEN 'BAJA'
                           WHEN i."PROBABILIDAD" = 2 THEN 'MEDIA'
                           WHEN i."PROBABILIDAD" = 3 THEN 'ALTA'
                           ELSE 'no definida' -- Opcional: para valores distintos de 1, 2 o 3
                         END,
    'PROBABILIDAD', i."PROBABILIDAD",
    'REGION', i."REGION",
    'DEPARTAMENTO', i."DEPARTAMENTO",
	  'FECHA_EJECUCION', i."FECHA_EJECUCION",
    'MUNICIPIO', i."MUNICIPIO"
  )
) AS geom
FROM incendios as i
INNER JOIN mgn_mpio_politico as mg
ON i."COD_DANE" = mg.mpio_cdpmp;

"""
    try:
        with engine.connect() as conn:
            conn.execute(create_view_sql)
            return("Vista creada exitosamente.")
    except SQLAlchemyError as e:
        return(f"Error al crear la vista: {e}")

In [13]:
vista_incendios(url_conexion)

'Vista creada exitosamente.'